In [2]:
import sys
import re
import json

In [3]:
# Check comment for severity and type
def analyze_comment(comment):
    negative_words = ['inefficient', 'bad', 'poor', 'wrong', 'terrible', 'unnecessary', 'redundant', 'vague']
    comment = comment.lower()
    severity = 'harsh' if any(word in comment for word in negative_words) else 'mild'

    if 'loop' in comment and 'inefficient' in comment:
        return severity, 'inefficient_loop'
    if 'variable' in comment and ('bad' in comment or 'vague' in comment):
        return severity, 'bad_variable_name'
    if 'boolean' in comment and 'redundant' in comment:
        return severity, 'redundant_boolean'
    if 'dictionary' in comment and ('unnecessary' in comment or 'redundant' in comment):
        return severity, 'redundant_dict'
    if 'string' in comment and 'inefficient' in comment:
        return severity, 'inefficient_string_concat'
    if 'error handling' in comment:
        return severity, 'missing_error_handling'
    if 'count' in comment and 'simplified' in comment:
        return severity, 'inefficient_count'
    if 'docstring' in comment:
        return severity, 'missing_docstring'
    if 'hardcoded' in comment:
        return severity, 'hardcoded_value'
    return severity, 'generic'

In [4]:
# Generate empathetic feedback for a comment
def generate_feedback(code, comment, severity, kind):
    templates={
        'inefficient_loop':{
            'rephrase':{
                'mild': "Nice logic! We can speed up those loops.",
                'harsh': "Good start! Let's make the loops faster."
            },
            'why': "Nested loops can be slow. Use sum() or list comprehensions for better performance. See [PEP 8](https://peps.python.org/pep-0008/).",
            'fix': """def process_user_data(records):
    return [{'user_id': user['user_id'], 'total_points': sum(point for point in user['points'] if point > 0)}
            for user in records if user['status'] and user['profile']['is_complete'] and sum(point for point in user['points'] if point > 0) >= 50]"""
        },
        'bad_variable_name':{
            'rephrase':{
                'mild': "Solid code! Clearer names would help.",
                'harsh': "Great base! Let's use better names."
            },
            'why': "Descriptive names improve readability. See [PEP 8](https://peps.python.org/pep-0008/#naming-conventions).",
            'fix': """def process_user_data(records):
    output_list = []
    for user in records:
        if user['status'] == True and user['profile']['is_complete'] == True:
            temp_dict = {}
            temp_dict['user_id'] = user['user_id']
            temp_dict['total_points'] = 0
            for i in range(len(user['points'])):
                if user['points'][i] > 0:
                    temp_dict['total_points'] = temp_dict['total_points'] + user['points'][i]
            if temp_dict['total_points'] >= 50:
                output_list.append(temp_dict)
    return output_list"""
        },
        'redundant_boolean':{
            'rephrase':{
                'mild': "Nice conditions! Drop extra boolean checks.",
                'harsh': "Good job! Let's simplify booleans."
            },
            'why': "Booleans don't need '== True' in Python. See [PEP 8](https://peps.python.org/pep-0008/#programming-recommendations).",
            'fix': """def process_user_data(records):
    output_list = []
    for x in records:
        if x['status'] and x['profile']['is_complete']:
            temp_dict = {}
            temp_dict['user_id'] = x['user_id']
            temp_dict['total_points'] = 0
            for i in range(len(x['points'])):
                if x['points'][i] > 0:
                    temp_dict['total_points'] = temp_dict['total_points'] + x['points'][i]
            if temp_dict['total_points'] >= 50:
                output_list.append(temp_dict)
    return output_list"""
        },
        'redundant_dict':{
            'rephrase':{
                'mild':"Good approach! Skip the temp dictionary.",
                'harsh':"Nice work! Let's avoid extra dictionaries."
            },
            'why':"Directly building results saves steps. See [Python Tips](https://docs.python.org/3/tutorial/datastructures.html).",
            'fix':"""def process_user_data(records):
    output_list = []
    for x in records:
        if x['status'] == True and x['profile']['is_complete'] == True:
            total = sum(x['points'][i] for i in range(len(x['points'])) if x['points'][i] > 0)
            if total >= 50:
                output_list.append({'user_id': x['user_id'], 'total_points': total})
    return output_list"""
        },
        'inefficient_string_concat': {
            'rephrase':{
                'mild': "Cool summary! Joining strings is faster.",
                'harsh': "Great summary! Let's speed up string building."
            },
            'why': "String concatenation in loops is slow; use join(). See [Python Strings](https://docs.python.org/3/library/stdtypes.html#str.join).",
            'fix': """def create_summary(data):
    lines = [f"User ID: {u['user_id']} Points: {u['total_points']}" for u in data]
    lines.append(f"Total Processed: {len(data)}")
    return '\n'.join(lines)"""
        },
        'missing_error_handling': {
            'rephrase': {
                'mild': "Solid code! Add error checks for safety.",
                'harsh': "Good effort! Let's handle errors."
            },
            'why': "Checking keys prevents crashes. See [Python Errors](https://docs.python.org/3/tutorial/errors.html).",
            'fix': """def process_user_data(records):
    output_list = []
    for x in records:
        try:
            if x['status'] == True and x['profile']['is_complete'] == True:
                temp_dict = {}
                temp_dict['user_id'] = x['user_id']
                temp_dict['total_points'] = 0
                for i in range(len(x['points'])):
                    if x['points'][i] > 0:
                        temp_dict['total_points'] = temp_dict['total_points'] + x['points'][i]
                if temp_dict['total_points'] >= 50:
                    output_list.append(temp_dict)
        except KeyError:
            continue
    return output_list"""
        },
        'inefficient_count':{
            'rephrase':{
                'mild': "Nice counting! We can simplify it.",
                'harsh': "Good count! Let's make it easier."
            },
            'why': "Use len() instead of manual counters. See [Python Built-ins](https://docs.python.org/3/library/functions.html#len).",
            'fix': """def create_summary(data):
    result = ''
    for u in data:
        result = result + 'User ID: ' + str(u['user_id']) + ' Points: ' + str(u['total_points']) + '\n'
    result = result + 'Total Processed: ' + str(len(data))
    return result"""
        },
        'missing_docstring':{
            'rephrase':{
                'mild': "Great functions! Docstrings would help.",
                'harsh': "Nice code! Let's add docstrings."
            },
            'why': "Docstrings clarify function purpose. See [PEP 257](https://peps.python.org/pep-0257/).",
            'fix': """def process_user_data(records):
    \"\"\"Filter users by status and profile, summing positive points.\"\"\"
    output_list = []
    for x in records:
        if x['status'] == True and x['profile']['is_complete'] == True:
            temp_dict = {}
            temp_dict['user_id'] = x['user_id']
            temp_dict['total_points'] = 0
            for i in range(len(x['points'])):
                if x['points'][i] > 0:
                    temp_dict['total_points'] = temp_dict['total_points'] + x['points'][i]
            if temp_dict['total_points'] >= 50:
                output_list.append(temp_dict)
    return output_list"""
        },
        'hardcoded_value':{
            'rephrase':{
                'mild': "Good logic! Flexible thresholds are better.",
                'harsh': "Solid code! Avoid hardcoding numbers."
            },
            'why': "Configurable values improve flexibility. See [PEP 8](https://peps.python.org/pep-0008/).",
            'fix': """def process_user_data(records, min_points=50):
    output_list=[]
    for x in records:
        if x['status']==True and x['profile']['is_complete'] == True:
            temp_dict={}
            temp_dict['user_id'] = x['user_id']
            temp_dict['total_points'] = 0
            for i in range(len(x['points'])):
                if x['points'][i] > 0:
                    temp_dict['total_points'] = temp_dict['total_points'] + x['points'][i]
            if temp_dict['total_points'] >= min_points:
                output_list.append(temp_dict)
    return output_list"""
        },
        'generic': {
            'rephrase':{
                'mild': "Nice job! Let's tweak this bit.",
                'harsh': "You're doing great! Let's refine this."
            },
            'why': "Clear code helps teams collaborate.",
            'fix': code
        }
    }

    template = templates.get(kind, templates['generic'])
    return {
        'positive_rephrasing': template['rephrase'][severity],
        'why': template['why'],
        'suggested_improvement': template['fix']
    }


In [5]:
# Generate encouraging summary
def generate_summary(feedback_list):
    return (
        "Great work on this code! These tips will make it faster and clearer. "
        "Keep it up, and you'll keep getting better!"
    )

In [6]:
# Process JSON and output Markdown report
def main():
    try:
        with open("sample.json", "r", encoding="utf-8") as f:
            data = json.load(f)
        code = data['code_snippet']
        comments = data['review_comments']
    except FileNotFoundError:
        print("Error: 'sample.json' not found.", file=sys.stderr)
        return
    except json.JSONDecodeError:
        print("Error: Invalid JSON in 'sample.json'.", file=sys.stderr)
        return
    except KeyError:
        print("Error: JSON needs 'code_snippet' and 'review_comments' keys.", file=sys.stderr)
        return

    feedback_list=[]
    for comment in comments:
        severity, kind = analyze_comment(comment)
        feedback = generate_feedback(code, comment, severity, kind)
        feedback_list.append((comment, feedback))

    print("# Code Review Feedback\n")
    for comment, feedback in feedback_list:
        print(f"### Comment: \"{comment}\"")
        print(f"* **Rephrasing:** {feedback['positive_rephrasing']}")
        print(f"* **Why:** {feedback['why']}")
        print("* **Suggestion:**")
        print(f"```python\n{feedback['suggested_improvement']}\n```")
        print("---")

    print("## Summary")
    print(generate_summary(feedback_list))

if __name__ == "__main__":
    main()

# Code Review Feedback

### Comment: "Nested loops make this inefficient."
* **Rephrasing:** Good start! Let's make the loops faster.
* **Why:** Nested loops can be slow. Use sum() or list comprehensions for better performance. See [PEP 8](https://peps.python.org/pep-0008/).
* **Suggestion:**
```python
def process_user_data(records):
    return [{'user_id': user['user_id'], 'total_points': sum(point for point in user['points'] if point > 0)}
            for user in records if user['status'] and user['profile']['is_complete'] and sum(point for point in user['points'] if point > 0) >= 50]
```
---
### Comment: "Variables 'x' and 'u' are too vague."
* **Rephrasing:** Great base! Let's use better names.
* **Why:** Descriptive names improve readability. See [PEP 8](https://peps.python.org/pep-0008/#naming-conventions).
* **Suggestion:**
```python
def process_user_data(records):
    output_list = []
    for user in records:
        if user['status'] == True and user['profile']['is_complete'] 